Model: 
- Sequential Model, Convolutional Neural Network 
- 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf # machine learning part
import keras
from keras import layers
from keras import ops

In [3]:
training_data = pd.read_csv('./TestData/train.csv')
x_train_dataframe = training_data.iloc[:, training_data.columns != 'label']
x_train = x_train_dataframe.values
y_train = training_data.label.values

testing_data = pd.read_csv('./TestData/test.csv')

x_test = testing_data.values


In [5]:
x_train = tf.keras.utils.normalize(x_train)
x_test = tf.keras.utils.normalize(x_test)

Convolutional Layers: 
- Each convolutional layer passes on a "filter" which is a nxn matrix
- The filter slides along the grid of pixel values, and the layer passes a new image with the product of the filter and each corresponding sliding window

Pooling Layers: 
- Similar sliding window idea as convolution, with the "stride" equal to the number of pixels that the filter passes along
- Computes an operation on each valid position, and computes a single value as output, usually max or average
- Reduces the size of an intermediate features

Dense Layers: 


In [34]:
model = tf.keras.models.Sequential()
model.add(keras.Input(shape=(28, 28, 1)))

#CNN 128 filter that produces the same size image
# Pooling layer by factor of 2

model.add(layers.Conv2D(128, 2, padding="same", activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2, 2)))

model.add(layers.Conv2D(64, 2, padding="same", activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2, 2)))

model.add(layers.Conv2D(32, 2, padding="same", activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2, 2)))

#
model.add(layers.Flatten())
model.add(layers.Dense(128, activation="relu"))
model.add(layers.Dense(128, activation="relu"))
model.add(layers.Dense(128, activation="relu"))
model.add(layers.Dense(128, activation="relu"))



model.add(layers.Dense(10, activation="softmax"))

In [36]:
model.compile(optimizer= 'adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

x_train = x_train.reshape(-1, 28, 28)

model.fit(x_train, y_train, shuffle=True, validation_split=0.2, validation_data=None, epochs=30)

model.save('CNN.model.keras')

saved_model = tf.keras.models.load_model('CNN.model.keras')

Epoch 1/30
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 16s 15ms/step - accuracy: 0.9917 - loss: 0.0279 - val_accuracy: 0.9820 - val_loss: 0.0784
Epoch 2/30
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 15s 15ms/step - accuracy: 0.9922 - loss: 0.0245 - val_accuracy: 0.9849 - val_loss: 0.0666
Epoch 3/30
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 16s 15ms/step - accuracy: 0.9938 - loss: 0.0218 - val_accuracy: 0.9839 - val_loss: 0.0643
Epoch 4/30
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 17s 16ms/step - accuracy: 0.9932 - loss: 0.0202 - val_accuracy: 0.9820 - val_loss: 0.0762
Epoch 5/30
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 18s 17ms/step - accuracy: 0.9946 - loss: 0.0185 - val_accuracy: 0.9861 - val_loss: 0.0605
Epoch 6/30
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 18s 18ms/step - accuracy: 0.9950 - loss: 0.0172 - val_accuracy: 0.9867 - val_loss: 0.0668
Epoch 7/30
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 18s 18ms/step - accuracy: 0.9955 - loss: 0.0139 - val_accuracy: 0.9845 - val_loss: 0.0812
Epoch 8/30
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 18s 17ms/step - accuracy: 0.9940 -

In [32]:
x_test = x_test.reshape(-1, 28, 28)
predictions = saved_model.predict(x_test)
predicted_labels = predictions.argmax(axis=1)

875/875 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step


In [33]:
results = pd.DataFrame({
    'ImageId': range(1, len(predicted_labels) + 1), 
    'Label': predicted_labels
})

results.to_csv('CNN_predictions.csv', index=False)